# Monte Carlo Simulation

Working with Lennard-Jones potential today. $$U(r) = 4\epsilon \left [ \left( \frac{\sigma}{r} \right )^{12} - \left( \frac{\sigma}{r} \right )^6 \right ]$$

Reduced equation: $$ U^*\left(r_{ij} \right) = 4 \left[\left(\frac{1}{r^*_{ij}}\right)^{12} -\left(\frac{1}{r^*_{ij}}\right)^{6} \right] $$


In [7]:
import math

In [8]:
def calculate_LJ(r_ij):
    """
    The LJ interaction energy between two particles.
    
    Computes the pairwise Lennard-Jones interaction energy based on the separation distance in reduced units.
    
    Parameters:
    ```````````
    r_ij : float
        The separation distance in reduced units.
    
    Returns:
    ````````
    pairwise energy : float
        The pairwise Lennard-Jones interaction energy in reduced units.
    """
    
    inverse = 1/r_ij
    pairwise_energy = 4 *(math.pow(inverse, 12) - math.pow(inverse, 6))
    return pairwise_energy

In [9]:
assert calculate_LJ(1) == 0
assert calculate_LJ(math.pow(2, 1/6)) == -1

In [22]:
a = 1
if a is not None:
    print('not none')

not none


In [69]:
def calculate_distance(coord1, coord2, box_length = None):
    """
    Calculate the distance between two points. When box_length is set, we use the the minimum image convention to calculate.
    
    Parameters:
    ```````````
    coord1, coord2 : list
        The atomic coordinates [x, y, z]
        
    box_length : float, optional
        The box length. The function assumes the box is a cube.
    
    Returns:
    ````````
    distance : float
        The distance between the two atoms.
    """
    distance = 0
    for i in range(len(coord1)):
        coord_dist = coord1[i] - coord2[i]
        if box_length:
            if abs(coord_dist) > box_length / 2:
                coord_dist = coord_dist - (box_length * round(coord_dist / box_length))
        distance += coord_dist**2
        
    distance = math.sqrt(distance)
    
    return distance

In [75]:
point1 = [0,0,0]
point2 = [0,0,8]

dist1 = calculate_distance(point1, point2, box_length = 10)
assert dist1 == 2

point3 = [0,0,0]
point4 = [0,1,1]

dist2 = calculate_distance(point3, point4)
assert dist2 == math.sqrt(2)

In [28]:
coordinates = [[0, 0, 0], [0, math.pow(2, 1/6), 0], [0, 2*math.pow(2, 1/6), 0]]

In [55]:
def calculate_total_energy(coords, cutoff):
    """
    Calculates the total interaction energy existing among a set of coordinates.
    
    Parameters:
    ```````````
    coords : list
        Nested list of coordinates [x,y,z]
    cutoff : float
        The cutoff distance for the system
        
    Returns:
    ````````
    total_energy : float
        The total interaction energy calculated from LJ potential.
    """
    
    total_energy = 0
    
    for i in range(len(coords)):
        for j in range(i+1, len(coords)):
            dist = calculate_distance(coords[i], coords[j], box_length)
            if dist < cutoff:
                energy = calculate_LJ(dist)
                total_energy += energy
    
    return total_energy

In [30]:
calculate_total_energy(coordinates, math.inf)

-2.031005859375

## Calculating energy from NIST data

In [18]:
import os

In [19]:
def read_xyz(filepath):
    """
    Reads coordinates from an xyz file.
    
    Parameters
    ----------
    filepath : str
       The path to the xyz file to be processed.
       
    Returns
    -------
    atomic_coordinates : list
        A two dimensional list containing atomic coordinates
    """
    
    with open(filepath) as f:
        box_length = float(f.readline().split()[0])
        num_atoms = float(f.readline())
        coordinates = f.readlines()
    
    atomic_coordinates = []
    
    for atom in coordinates:
        split_atoms = atom.split()
        
        float_coords = []
        
        # We split this way to get rid of the atom label.
        for coord in split_atoms[1:]:
            float_coords.append(float(coord))
            
        atomic_coordinates.append(float_coords)
        
    
    return atomic_coordinates, box_length


In [49]:
filename = os.path.join('lj_sample_configurations', 'lj_sample_config_periodic1.txt')
atomic_coordinates, box_length = read_xyz(filename)

In [74]:
assert math.isclose(calculate_total_energy(atomic_coordinates, 3), -4.3515E+03, rel_tol = 0.01)